In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

Probability density function

In [ ]:
def f(x): return 0.5 * np.exp(-np.abs(x))

Metropolis-Hastings algorithm

In [ ]:
def MetropolisHastings(N, s, x0):
    samples = [x0]
    xi = x0
    
    for i in range(1, N):
        xp = np.random.normal(loc=samples[i-1], scale=s)
        
        # Compute acceptance ratio
        r = f(xp) / f(samples[i-1])
        
        # Generate a random number from uniform distribution
        u = np.random.uniform(0, 1)
        
        # Accept or reject the proposal
        if np.log(u) < np.log(r): xi = xp
        
        samples.append(xi)
    
    return np.array(samples)

Mean of chain j

In [ ]:
def Mj(N, xj):
    xj = xj.flatten()
    jMean = 0.0
    for i in range(1, N): jMean += xj[i]
    return jMean/N

Varience of chain j

In [ ]:
def Vj(N, xj, mj):
    xj = xj.flatten()
    jVar = 0.0
    for i in range(1, N): jVar += ( xj[i] - mj )**2
    return jVar/N

Varience of all

In [ ]:
def W(J, vj):
    ovrVar = 0.0
    for i in range(J): ovrVar += vj[i]
    return ovrVar/J

Mean of all

In [ ]:
def M(J, mj):
    ovrMean = 0.0
    for i in range(J): ovrMean += mj[i]
    return ovrMean/J

Varience between samples

In [ ]:
def B(J, M, mj):
    betVar = 0.0
    for i in range(J): betVar += ( mj[i] - M )**2
    return betVar

R value

In [ ]:
def RVal(Bs, Ws):
    return np.sqrt( (Bs+Ws)/Ws )

Convergence

In [ ]:
def ConvergenceTest(chains):
    meanOfChains = []
    varOfChains = []
    sampleVar = 0.0
    sampleMean = 0.0
    betSampleVar = 0.0
    for i in range(J):
        meanOfChains.append( Mj(N, chains[i]) )
        varOfChains.append( Vj(N, chains[i], meanOfChains[i]) )
    sampleVar = W(J, varOfChains)
    sampleMean = M(J, meanOfChains)
    betSampleVar = B(J, sampleMean, meanOfChains)
    R = RVal(betSampleVar, sampleVar)
    return R, sampleMean, sampleVar, betSampleVar

Curve Fit

In [ ]:
def cfMod(x, a, b, c): return a * np.exp(-b*x) + c

Parameters

In [ ]:
J = 4
N = 2000
s = 0.001
x0 = 0.5

In [ ]:
chains = []
for i in range(1,J+1): chains.append( MetropolisHastings(N, s, x0) )
returns = ConvergenceTest(chains)
print("Outputs for N=2000, J=4 and s=0.001:")
print( "M value: ", returns[1])
print( "W value: ", returns[2])
print( "B value: ", returns[3])
print( "R value: ", returns[0])

In [ ]:
s = np.linspace(0.001, 1, 100)
rvalues = []

In [ ]:
for i in s:
    chains = []
    for j in range(1,J+1): chains.append( MetropolisHastings(N, i, x0) )
    returns = ConvergenceTest(chains)
    rvalues.append(returns[0])

In [ ]:
param, param_cov = curve_fit(cfMod, s, rvalues)
rfit = cfMod(s, param[0], param[1], param[2])

In [ ]:
fig, ax = plt.subplots()
plt.plot(s, rvalues, 'ro', label='R Values')
plt.plot(s, rfit, '-b', label='R Values (Curve Fitted)')
plt.title('Standard Deviation and R Value Plot')
plt.xlabel('Standard Deviation')
plt.ylabel('R Value')
plt.legend()
plt.show()